# Desafio - Explorando IA generativa em um pipeline de ETL com Python

In [6]:
import pandas as pd
import json
import requests

In [ ]:
#Repositório da API do desafio: https://github.com/digitalinnovationone/santander-dev-week-2023-api
sdw2023_api_url = 'https://sdw-2023-prd.up.railway.app/'

## Ponto inicial da planilha

In [7]:
df = pd.read_csv('Planilha_simples.csv')
df

,UserID
0,1
1,2
2,3
3,4
4,5


In [11]:
user_ids = df['UserID'].tolist()
user_ids

[1, 2, 3, 4, 5]

## Extract

In [12]:
def get_user(id):
  response = requests.get(f'{sdw2023_api_url}/users/{id}')
  return response.json() if response.status_code == 200 else None

users = [user for id in user_ids if (user := get_user(id)) is not None]
print(json.dumps(users, indent=2))

NameError: name 'requests' is not defined

In [ ]:
openai_api_key = 'TODO'

## Transform

In [ ]:
import openai

openai.api_key = openai_api_key

def generate_ai_news(user):
  completion = openai.ChatCompletion.create(
    model="gpt-4",
    messages=[
      {
          "role": "system",
          "content": "Você é um especialista em markting bancário."
      },
      {
          "role": "user",
          "content": f"Crie uma mensagem para {user['name']} sobre a importância dos investimentos (máximo de 100 caracteres)"
      }
    ]
  )
  return completion.choices[0].message.content.strip('\"')

for user in users:
  news = generate_ai_news(user)
  print(news)
  user['news'].append({
      "icon": "https://digitalinnovationone.github.io/santander-dev-week-2023-api/icons/credit.svg",
      "description": news
  })

## Load

In [ ]:
def update_user(user):
  response = requests.put(f"{sdw2023_api_url}/users/{user['id']}", json=user)
  return True if response.status_code == 200 else False

for user in users:
  success = update_user(user)
  print(f"User {user['name']} updated? {success}!")